In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload

# reload from external changes

import ml_util
ml_util = reload(ml_util)
from ml_util import *

import datasets
datasets = reload(datasets)

import generative
generative = reload(generative)

import cnn
cnn = reload(cnn)

import adversarial_generation
adversarial_generation = reload(adversarial_generation)
from adversarial_generation import *

import evaluation_util
evaluation_util = reload(evaluation_util)
from evaluation_util import EvaluationUtil

Util.configure(5000)

### Load a dataset and classifiers

In [2]:
#dataset_info = DatasetInfo.MNIST
dataset_info = DatasetInfo.CelebA128Gender
#dataset_info = DatasetInfo.LSUN128

class_proportions = None
if dataset_info in [DatasetInfo.CelebA128Gender, DatasetInfo.LSUN128]:
    no_classes = 2
    if dataset_info == DatasetInfo.CelebA128Gender:
        ds = datasets.CelebAData(20)
        classifier_d = "celeba-128"
        class_proportions = np.array([0.583, 0.417])
    elif dataset_info == DatasetInfo.LSUN128:
        ds = datasets.LSUNData()
        classifier_d = "lsun-128"
    gm_loader = lambda label: generative.PIONEER(dataset_info, label, ds, spectral_norm_warming_no_images=25)
elif dataset_info == DatasetInfo.MNIST:
    no_classes = 10
    ds = datasets.MNISTData()
    classifier_d = "mnist"
    gm_loader = lambda label: generative.WGAN(dataset_info, label, ds)
else:
    raise RuntimeError()
if class_proportions is None:
    class_proportions = np.repeat(1 / no_classes, no_classes)
    
def load_classifier(weights_filename: str):
    c = cnn.Trainer(classifier_d, ds.get_train_loader, ds.get_test_loader)
    c.restore_params_from_disk(weights_filename)
    return c

nonrobust_classifier = load_classifier(f"new_classifiers/plain_{dataset_info.name}.bin")
robust_classifier = load_classifier(f"new_classifiers/robust_{dataset_info.name}.bin")
classifiers = [nonrobust_classifier, robust_classifier]

def advgen_experiments(adversary: Adversary, noise_eps: float, total_no_images: int):
    decay_factor = EpsDTransformer().eps_to_d(noise_eps)
    for use_generated_images in [False, True]:
        advgen = AdversarialGenerator(None, classifiers, use_generated_images, decay_factor)
        no_images = np.round(class_proportions * total_no_images)
        for i in range(no_classes):
            LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i}, "
                         f"{'GENERATED' if use_generated_images else 'RECONSTRUCTED'} ***")
            LogUtil.info(f"noise_epsilon = {noise_eps:.5f}, decay_factor = {decay_factor:.5f}")
            gm = gm_loader(i)
            advgen.set_generative_model(gm)
            advgen.generate(adversary, int(no_images[i]), True, clear_stat=(i == 0))
            gm.destroy()
        LogUtil.info("*** STATISTICS ***")
        advgen.print_stats(True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
celeba-128 classifier: 300138 trainable parameters
celeba-128 classifier: 300138 trainable parameters


In [ ]:
for i in range(no_classes):
    LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i} ***")
    gm = gm_loader(i)
    EvaluationUtil.show_reconstructed_images(gm, 1, 5)
    EvaluationUtil.show_generated_images(gm, 1, 10)
    # add perturbations and see classification outcomes + calculate classification accuracy
    RandomPerturbationStatistician(gm, classifiers, 1, 1, True, np.linspace(0.25, 1.00, 4)).process()
    # produce "class mean" vector
    advgen = AdversarialGenerator(gm, classifiers, True, 1.0)
    advgen.generate(PGDAdversary(0.01, 1, 0.1, False, 0, verbose=0), 1, 1, False)
    gm.destroy()

### Measure resistance to noise

In [ ]:
for i in range(no_classes):
    LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i} ***")
    gm = gm_loader(i)
    RandomPerturbationStatistician(gm, classifiers, 3, 200, True, np.linspace(0.25, 1.00, 4)).process()
    gm.destroy()

### Find latent adversarial perturbations

In [ ]:
# search of minimum adversarial perturbations
max_rho, noise_eps = 2.5, 1.0
if dataset_info == DatasetInfo.MNIST:
    adversary = PGDAdversary(max_rho, 50, 0.05, True, 0, verbose=0, n_repeat=12, repeat_mode="min")
else:
    adversary = PGDAdversary(max_rho, 1250, 0.002, False, 0, verbose=0)
advgen_experiments(adversary, noise_eps, 10)